In [1]:
import os
os.chdir('../..')

In [158]:
%load_ext autoreload
%autoreload 2

import sklearn.model_selection
import pandas as pd
from sklearn.metrics import accuracy_score
from flair.models import TARSClassifier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [95]:
df = pd.read_json('data/papers_dbpedia_affiliations.json', orient='index', convert_dates=False, convert_axes=False)
df = df[df['manually_annotated']]
df['affs'] = df.affiliations.apply(lambda a: list(zip(a['names'], a['types'])))

In [96]:
df_parsed = df.explode('affs')
df_parsed = df_parsed['affs'].apply(pd.Series)
df_parsed = df_parsed.dropna()

X, y = df_parsed[0], df_parsed[1]
y = y.replace('nonprofit', 'academic')

In [103]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, train_size=.3, stratify=y, random_state=123)

In [154]:
from flair.models import TARSClassifier
from flair.data import Sentence

tars_zero_shot = TARSClassifier.load('tars-base')
tars_zero_shot.add_and_switch_to_new_task("ACADEMIC_COMPANY", label_dictionary=classes,label_type='affiliation', multi_label=False)                  

def predict_zero_shot(texts, classes):
    prepared = texts.apply(Sentence)
    for text in prepared:
        tars_zero_shot.predict(text,label_threshold=0.0)
    return prepared


def get_predictions(sentences: pd.Series):
    labels = sentences.apply(lambda s: sorted(s.labels, key=lambda l: l.score, reverse=True)[0].value)
    scores = sentences.apply(lambda s: sorted(s.labels, key=lambda l: l.score, reverse=True)[0].score)
    
    return labels, scores


classes = list(y.drop_duplicates())

2022-01-08 19:28:15,095 loading file /home/sgizinski/.flair/models/tars-base-v8.pt


In [159]:
sentences = predict_zero_shot(X_test, classes)
pred, scores = get_predictions(sentences)
acc = accuracy_score(y_test, pred)

In [165]:
acc

0.9375

In [162]:
y_test

1909.03723    academic
2009.07708    academic
2009.09072     company
1807.10478    academic
1509.06321    academic
                ...   
2003.01504    academic
1909.02827     company
1907.05131    academic
1706.09773    academic
2004.08886    academic
Name: 1, Length: 64, dtype: object

In [163]:
pred

1909.03723    academic
2009.07708    academic
2009.09072     company
1807.10478    academic
1509.06321    academic
                ...   
2003.01504    academic
1909.02827     company
1907.05131     company
1706.09773    academic
2004.08886    academic
Name: 0, Length: 64, dtype: object

# TODO: porównać z metodami opartymi o 
* słowa kluczowe (university, etc.)
* maile
* co tam jeszcze było
